# 建議先從rag-faiss.ipynb開始跑 環境最複雜 但是弄完 4份都可以跑

事前準備:

python 3

VScode商城:

python

jupyter

import和拿key

沒裝openai_SDK使用:```pip install openai```

沒裝tabulate使用:```pip install tabulate```

沒設定key使用:```setx OPENAI_API_KEY “<yourkey>”```

```echo %OPENAI_API_KEY%```可以看你的KEY

In [1]:
import os
from openai import OpenAI
client = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')

# input在此輸入

In [2]:
input = "hey flora! Im having some issue with password! can you reset it for me?"

創assistant,thread,message!

In [4]:
from tabulate import tabulate

# Upload a file with an "assistants" purpose
file = client.files.create(
    file=open("data\manual.csv", "rb"),
    purpose='assistants'
)

assistant = client.beta.assistants.create(
    name="Flora Customer Service",
    instructions="You are customer service from Flora.You can only answer questions related to Flora. when answering any question, you must always answer base on manual which user have provided and respond only once.",
    tools=[{"type": "retrieval"},{"type": "code_interpreter"}],
    model="gpt-3.5-turbo-1106",
    file_ids=[file.id]
)

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=input
)


沒裝tabulate很醜

tabulate文件:https://pypi.org/project/tabulate/

表示assistant,thread,message內容

In [5]:
print(tabulate(assistant,headers=['assistant','content'],tablefmt="simple_outline"))

┌──────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ assistant    │ content                                                                                                                                                                                                │
├──────────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ id           │ asst_ldHnsqgKrTWD2r85rNYoDfvc                                                                                                                                                                          │
│ created_at   │ 1706252656                                                                                                     

In [6]:
print(tabulate(thread,headers=['thread','content'],tablefmt="simple_outline"))

┌────────────┬─────────────────────────────────┐
│ thread     │ content                         │
├────────────┼─────────────────────────────────┤
│ id         │ thread_bb20fpoijovFLyaJQMia7ooW │
│ created_at │ 1706252657                      │
│ metadata   │ {}                              │
│ object     │ thread                          │
└────────────┴─────────────────────────────────┘


In [7]:
print(tabulate(message,headers=['message','content'],tablefmt="simple_outline"))

┌──────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ message      │ content                                                                                                                                       │
├──────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ id           │ msg_BsFr5IO0MlVXGNPZqpI17Llx                                                                                                                  │
│ assistant_id │                                                                                                                                               │
│ content      │ [MessageContentText(text=Text(annotations=[], value='hey flora! Im having some issue with password! can you reset it for me?'), type='text')] │
│ created_at   │ 1706252657       

這次的run

可以使用新instruction複寫assisstant裡的instruction!

like:

```
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)
```


In [8]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

查看run的狀態!

以上如果還沒跑完會顯示queued!

以上如果跑完會顯示completed!

In [9]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
print(tabulate(run,headers=['run','content'],tablefmt='simple_outline'))

┌─────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ run             │ content                                                                                                                                                                                                │
├─────────────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ id              │ run_9K8ewl009Tz33Hv6LE49DU41                                                                                                                                                                           │
│ assistant_id    │ asst_ldHnsqgKrTWD2r85rNYoDfvc                                                                   

跑完後打開查看history messages!

In [12]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id,
  order='asc'
)
for i in messages.data:
    print(tabulate(i,tablefmt='simple_outline'))

┌──────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ id           │ msg_BsFr5IO0MlVXGNPZqpI17Llx                                                                                                                  │
│ assistant_id │                                                                                                                                               │
│ content      │ [MessageContentText(text=Text(annotations=[], value='hey flora! Im having some issue with password! can you reset it for me?'), type='text')] │
│ created_at   │ 1706252657                                                                                                                                    │
│ file_ids     │ []                                                                                                                                            │
│ metadata     │ {}               

最後一個訊息顯示

In [13]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
print(messages.data[0].content[0].text.value)

It seems that the file you uploaded contains some customer queries and responses related to using the Flora app, but it doesn't seem to contain any account information or password reset instructions. 

In order to reset your password, please follow the official password reset process provided by Flora. If you require further assistance, I recommend reaching out to Flora's customer support team for direct help with your account.
